In [ ]:
%load_ext autotime

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Dense, Embedding, LSTM, Dropout, Activation, Bidirectional
from sklearn.model_selection import train_test_split
#from keras.utils.np_utils import to_categorical
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.models import load_model
import re
import matplotlib.pyplot as plt
import logging

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import gensim

import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
import numpy as np
import copy
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import stats
import seaborn as sns
import pandas as pd
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [ ]:
df_annotations=  pd.read_csv('full_cleaned_dataset.csv')
data = df_annotations[['text','label']]

In [ ]:
def get_vocab_size_val(data,label):
    X_train, X_test, y_train, y_test = train_test_split(data,label , test_size=0.3, random_state=42, stratify=label)
    X_train, X_val, y_train, y_val = train_test_split(X_train,y_train , test_size=0.2, random_state=42, stratify=y_train)
    t = Tokenizer()
    t.fit_on_texts(X_train)
    vocab_size = len(t.word_index) + 1
    return vocab_size
def get_vocab_size(data,label):
    X_train, X_test, y_train, y_test = train_test_split(data,label , test_size=0.3, random_state=42, stratify=label)
    t = Tokenizer()
    t.fit_on_texts(X_train)
    vocab_size = len(t.word_index) + 1
    return vocab_size

def one_hot_enc(data, label, max_length):
    X_train, X_test, y_train, y_test = train_test_split(data,label , test_size=0.3, random_state=42, stratify=label)
    X_train, X_val, y_train, y_val = train_test_split(X_train,y_train , test_size=0.2, random_state=42, stratify=y_train)
    #print(y_test)
    t = Tokenizer()
    t.fit_on_texts(X_train)
    vocab_size = len(t.word_index) + 1
    #print(vocab_size)
    
    train_seq_nopad = t.texts_to_sequences(X_train)
    val_seq_nopad  = t.texts_to_sequences(X_val)
    test_seq_nopad  = t.texts_to_sequences(X_test)

    padded_train = pad_sequences(train_seq_nopad, maxlen=max_length, padding='post')
    padded_val = pad_sequences(val_seq_nopad, maxlen=max_length, padding='post')
    padded_test = pad_sequences(test_seq_nopad, maxlen=max_length, padding='post')
    
    return padded_train, padded_val,padded_test, vocab_size
    
def change_y(y):
    values = np.array(y)
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(values)
    #print(integer_encoded)
    onehot_encoder = OneHotEncoder(sparse=False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
    return onehot_encoded
    


In [ ]:
onehot_encoded=change_y(data['label'])
vocab_size_val= get_vocab_size_val(data['text'],data['label'])
vocab_size=get_vocab_size(data['text'],data['label'])

In [ ]:
def bidir_LSTM_1_hidden_layer_lr( model_name, name_accplot, name_lossplot, name_acc_csv, name_loss_csv,  lr, y, data, emb_len, max_length, lstm_out, batch_size, epochs, kernel_reg= 1e-3, rec_reg=1e-3, bias_reg=1e-3, act_reg=1e-3, drop_out=0.2, rec_drop=0.2):
    X_train, X_test, y_train, y_test = train_test_split(data, onehot_encoded, test_size=0.3, random_state=42, stratify=y)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42, stratify=y_train) 
    X_train, X_val, X_test, vocab_size=one_hot_enc(data, y, 500)
    #print(y_test)
    #model
    e = Embedding(vocab_size, emb_len, mask_zero=True, input_length=max_length)
    model= Sequential()
    model.add(e)
    model.add(Bidirectional(LSTM(lstm_out, dropout=drop_out, recurrent_dropout= rec_drop, kernel_regularizer=l2(kernel_reg),
            recurrent_regularizer= l2(rec_reg),bias_regularizer=l2(bias_reg), activity_regularizer=l2(act_reg))))#return_sequences=True
    model.add(Dense(units=3, activation='softmax'))
    optimizer= Adam(learning_rate=lr)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['acc'])
    print(' the model looks like this for the following parameters: the vocabulary size is ', vocab_size, 'the embedded_length is ', emb_len, 'the maximum length is ',max_length, 'the number of hidden nodes is ', lstm_out, 'The batch size is ',  batch_size, 'the number of epochs is ', epochs, 'the dropout of the dropout layer is ', drop_out )

    callback = LearningRateScheduler(scheduler, verbose=1)
    es= EarlyStopping(monitor='val_acc', mode='max', verbose=1, patience=30)
    mc = ModelCheckpoint(filepath=model_name, monitor='val_acc', mode='max', verbose=1, save_best_only=True, save_freq='epoch')
    history=model.fit(X_train, y_train, batch_size=batch_size, callbacks=[callback, es, mc], epochs=epochs, validation_data=(X_val, y_val))
    print(history.history.keys())
    
    # summarize history for accuracy
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    fig1 = plt.gcf()
    fig1.savefig(name_accplot)
    plt.show()
    
    
    df1=pd.DataFrame()
    df1['acc']=history.history['acc']
    df1['val_acc']=history.history['val_acc']
    df1.to_csv(name_acc_csv)
    
    
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    fig1 = plt.gcf()
    fig1.savefig(name_lossplot)
    plt.show()
    
    
    df2=pd.DataFrame()
    df2['loss']=history.history['loss']
    df2['val_loss']=history.history['val_loss']
    df2.to_csv(name_loss_csv)


    return model


In [ ]:
from tensorflow import math
def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    elif lr <=1e-5:
        return lr
    else:
        return float(lr * np.exp(-0.1))

In [ ]:
lr=[1e-5,1e-6,1e-7]
for i in lr:
    bidir_LSTM_1_hidden_layer_lr('nodes_32_model.hdf5','nodes_32_acc.png', 'nodes_32_loss.png', 'nodes_32_acc.csv', 'nodes_32_loss.csv',i, onehot_encoded, data['text'], 256, 500,16, 1024,10)

In [ ]:
nodes=[[16,'nodes_16_model.hdf5','nodes_16_acc.png', 'nodes_16_loss.png', 'nodes_16_acc.csv', 'nodes_16_loss.csv']b]
for h,i,j,k,l,m in nodes:
    bidir_LSTM_1_hidden_layer_lr(i,j, k, l, m,1e-5, onehot_encoded,  data['text'], 256, 500,h, 1024,1000, 1e-2)

16 nodes gave 59% validation accuracy

In [ ]:
nodes= [[32,'nodes_32_model.hdf5','nodes_32_acc.png', 'nodes_32_loss.png', 'nodes_32_acc.csv', 'nodes_32_loss.csv']]
for h,i,j,k,l,m in nodes:
    bidir_LSTM_1_hidden_layer_lr(i,j, k, l, m,1e-4, onehot_encoded,  data['text'], 256, 500,h, 1024,1000, 1e-2)

In [ ]:
nodes: [[64,'nodes_64_model.hdf5','nodes_64_acc.png', 'nodes_64_loss.png', 'nodes_64_acc.csv', 'nodes_64_loss.csv'],[128,'nodes_128_model.hdf5','nodes_128_acc.png', 'nodes_128_loss.png', 'nodes_128_acc.csv', 'nodes_128_loss.csv']]
for h,i,j,k,l,m in nodes:
    bidir_LSTM_1_hidden_layer_lr(i,j, k, l, m,1e-5, onehot_encoded,  data['text'], 256, 500,h, 1024,1000, 1e-2)

We decided to go for 12 hidden nodes after the previous experiments

Parameter tuning

In [ ]:
reg=[[1e-2,'kern_1e-2_model.hdf5','kern_reg1e-2_acc.png', 'kern_reg1e-2_loss.png', 'kern_reg1e-2_acc.csv', 'kern_reg1e-2_loss.csv'],[1e-3,'kern_1e-3_model.hdf5','kern_reg1e-3_acc.png', 'kern_reg1e-3_loss.png', 'kern_reg1e-3_acc.csv', 'kern_reg1e-3_loss.csv'], [1e-4,'kern_1e-4_model.hdf5','kern_reg1e-4_acc.png', 'kern_reg1e-4_loss.png', 'kern_reg1e-4_acc.csv', 'kern_reg1e-4_loss.csv'], [1e-5,'kern_1e-5_model.hdf5','kern_reg1e-5_acc.png', 'kern_reg1e-5_loss.png', 'kern_reg1e-5_acc.csv', 'kern_reg1e-5_loss.csv']]
for h, i, j, k, l , m in reg:
    padded_docs=one_hot_enc(vocab_size_val, data['text'], 500)
    bidir_LSTM_1_hidden_layer_lr(i,j, k, l, m,1e-4, onehot_encoded, padded_docs, vocab_size_val, 256, 500,12, 1024, 1000,h)
    #iets toevoegen waarbij ik de accuraatheid op de validatieset dan kan displayen en vergelijken over de verschillende modellen

Evaluation

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(padded_docs, onehot_encoded, test_size=0.3, random_state=42, stratify=onehot_encoded)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42, stratify=y_train)
list_acc=[]
for i in ['kern_1e-2_model.hdf5','kern_1e-3_model.hdf5', 'kern_1e-4_model.hdf5', 'kern_1e-5_model.hdf5']:
    my_model = load_model(i)
    loss, acc = my_model.evaluate(X_val, y_val)
    print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))
    list_acc.append(acc)

In [ ]:
import matplotlib.pyplot as plt
reg=[1e-2, 1e-3, 1e-4, 1e-5]
plt.xlabel('Kernel Regularizer')
plt.ylabel('Accuracy')
plt.plot(reg, list_acc)
plt.show()


In [ ]:
reg=[[1e-2,'rec_1e-2_model.hdf5','rec_reg1e-2_acc.png', 'rec_reg1e-2_loss.png', 'rec_reg1e-2_acc.csv', 'rec_reg1e-2_loss.csv' ], [1e-3,'rec_1e-3_model.hdf5','rec_reg1e-3_acc.png', 'rec_reg1e-3_loss.png', 'rec_reg1e-3_acc.csv', 'rec_reg1e-3_loss.csv'],[1e-4,'rec_1e-4_model.hdf5','rec_reg1e-4_acc.png', 'rec_reg1e-4_loss.png', 'rec_reg1e-4_acc.csv', 'rec_reg1e-4_loss.csv'], [1e-5,'rec_1e-5_model.hdf5','rec_reg1e-5_acc.png', 'rec_reg1e-5_loss.png', 'rec_reg1e-5_acc.csv', 'rec_reg1e-5_loss.csv']]
padded_docs=one_hot_enc(vocab_size_val, data['text'], 500)
for h,i,j,k,l,m in reg:
    bidir_LSTM_1_hidden_layer_lr(i,j, k, l, m,1e-4, onehot_encoded, padded_docs, vocab_size_val, 256, 500,12, 1024, 1000,1e-2, h) 

In [ ]:
reg=[[1e-2,'bias_1e-2_model.hdf5','bias_reg1e-2_acc.png', 'bias_reg1e-2_loss.png', 'bias_reg1e-2_acc.csv', 'bias_reg1e-2_loss.csv' ], [1e-3,'bias_1e-3_model.hdf5','bias_reg1e-3_acc.png', 'bias_reg1e-3_loss.png', 'bias_reg1e-3_acc.csv', 'bias_reg1e-3_loss.csv'],[1e-4,'bias_1e-4_model.hdf5','bias_reg1e-4_acc.png', 'bias_reg1e-4_loss.png', 'bias_reg1e-4_acc.csv', 'bias_reg1e-4_loss.csv'], [1e-5,'bias_1e-5_model.hdf5','bias_reg1e-5_acc.png', 'bias_reg1e-5_loss.png', 'bias_reg1e-5_acc.csv', 'bias_reg1e-5_loss.csv']]
for h,i,j,k,l,m in reg:
    bidir_LSTM_1_hidden_layer_lr(i, j, k, l, m,1e-4, onehot_encoded, data['text'], 256, 500,12, 1024, 1000,1e-2, 1e-3, h) 
    #alle regularizers staan op 1e-3


In [ ]:
list_acc=[]
for i in['bias_1e-2_model.hdf5', 'bias_1e-3_model.hdf5', 'bias_1e-4_model.hdf5', 'bias_1e-5_model.hdf5']:
    my_model=load_model(i)
    loss, acc = my_model.evaluate(X_val, y_val)
    print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))
    list_acc.append(acc)

In [ ]:
import matplotlib.pyplot as plt
reg=[1e-2, 1e-3, 1e-4, 1e-5]
plt.xlabel('Kernel Regularizer')
plt.ylabel('Accuracy')
plt.plot(reg, list_acc)
plt.show()

In [ ]:
reg=[[1e-2,'act_1e-2_model.hdf5','act_reg1e-2_acc.png', 'act_reg1e-2_loss.png', 'act_reg1e-2_acc.csv', 'act_reg1e-2_loss.csv' ], [1e-3,'act_1e-3_model.hdf5','act_reg1e-3_acc.png', 'act_reg1e-3_loss.png', 'act_reg1e-3_acc.csv', 'act_reg1e-3_loss.csv'],[1e-4,'act_1e-4_model.hdf5','act_reg1e-4_acc.png', 'act_reg1e-4_loss.png', 'act_reg1e-4_acc.csv', 'act_reg1e-4_loss.csv'], [1e-5,'act_1e-5_model.hdf5','act_reg1e-5_acc.png', 'act_reg1e-5_loss.png', 'act_reg1e-5_acc.csv', 'act_reg1e-5_loss.csv']]
for h,i,j,k,l,m in reg:
    bidir_LSTM_1_hidden_layer_lr(i, j, k, l, m,1e-4, onehot_encoded, data['text'], 256, 500,12, 1024, 1000,1e-2, 1e-3,1e-5, h) 
    #alle regularizers staan op 1e-3

In [ ]:
list_acc=[]
for i in['act_1e-2_model.hdf5', 'act_1e-3_model.hdf5', 'act_1e-4_model.hdf5', 'act_1e-5_model.hdf5']:
    my_model=load_model(i)
    loss, acc = my_model.evaluate(X_val, y_val)
    print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))
    list_acc.append(acc)

In [ ]:
import matplotlib.pyplot as plt
reg=[1e-2, 1e-3, 1e-4, 1e-5]
plt.xlabel('Activity Regularizer')
plt.ylabel('Accuracy')
plt.plot(reg, list_acc)
plt.show()

In [ ]:
reg=[[0.1,'dropout_0.1_model.hdf5','dropout_0.1_acc.png', 'dropout_0.1_loss.png', 'dropout_0.1_acc.csv', 'dropout_0.1_loss.csv' ], [0.3,'dropout_0.3_model.hdf5','dropout_0.3_acc.png', 'dropout_0.3_loss.png', 'dropout_0.3_acc.csv', 'dropout_0.3_loss.csv']]
for h,i,j,k,l,m in reg:
    bidir_LSTM_1_hidden_layer_lr(i, j, k, l, m,1e-4, onehot_encoded, data['text'], 256, 500,12, 1024, 1000,1e-2, 1e-3,1e-5,1e-3, h) 

In [ ]:
list_acc=[]
for i in['dropout_0.1_model.hdf5', 'act_1e-3_model.hdf5', 'dropout_0.3_model.hdf5']:
    my_model=load_model(i)
    loss, acc = my_model.evaluate(X_val, y_val)
    print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))
    list_acc.append(acc)

In [ ]:
import matplotlib.pyplot as plt
reg=[0.1, 0.2, 0.3]
plt.xlabel('Dropout')
plt.ylabel('Accuracy')
plt.plot(reg, list_acc)
plt.show()

In [ ]:
reg=[[0.1,'rec_dropout_0.1_model.hdf5','rec_dropout_0.1_acc.png', 'rec_dropout_0.1_loss.png', 'rec_dropout_0.1_acc.csv', 'rec_dropout_0.1_loss.csv' ], [0.3,'rec_dropout_0.3_model.hdf5','rec_dropout_0.3_acc.png', 'rec_dropout_0.3_loss.png', 'rec_dropout_0.3_acc.csv', 'rec_dropout_0.3_loss.csv']]
for h,i,j,k,l,m in reg:
    bidir_LSTM_1_hidden_layer_lr(i, j, k, l, m,1e-4, onehot_encoded, data['text'], 256, 500,12, 1024, 1000,1e-2, 1e-3,1e-5,1e-3,0.2, h) 

In [ ]:
list_acc=[]
for i in['rec_dropout_0.1_model.hdf5', 'rec_dropout_0.3_model.hdf5']:
    my_model=load_model(i)
    loss, acc = my_model.evaluate(X_val, y_val)
    print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))
    list_acc.append(acc)

In [ ]:
import matplotlib.pyplot as plt
reg=[0.1, 0.3]
plt.xlabel(' Recurrent Dropout')
plt.ylabel('Accuracy')
plt.plot(reg, list_acc)
plt.show()

Evaluation best model

In [ ]:
my_model=load_model('rec_dropout_0.3_model.hdf5')
loss, acc = my_model.evaluate(X_test, y_test)


In [ ]:
predictions=my_model.predict(X_test)

In [ ]:
print(predictions)

In [ ]:
predictions=np.argmax(predictions, axis=1)

In [ ]:
predictions

In [ ]:
print(y_test)

In [ ]:
ground_truth=np.argmax(y_test,axis=1)

In [ ]:
print(ground_truth)

In [ ]:
from sklearn import metrics

In [ ]:
print('The Accuracy of the model is',metrics.accuracy_score(predictions, ground_truth))

In [ ]:
def cf_mat(y_test, y_pred):
    mat=metrics.confusion_matrix(y_test, y_pred)/len(y_test)
    return sns.heatmap(mat, annot=True)

In [ ]:
from sklearn import metrics
print(' The Macro F1 measure is', metrics.f1_score(ground_truth, predictions, average='macro'))
print(' The Macro precision is',metrics.precision_score(ground_truth, predictions, average='macro'))
print(' The Macro recall is',metrics.recall_score(ground_truth, predictions, average='macro'))
        
print('The Accuracy of the model is',metrics.accuracy_score(ground_truth, predictions))
print('The Spearman correlation of the model is ', stats.spearmanr(ground_truth, predictions))
print('The Kendall tau of the model is ', stats.kendalltau(ground_truth, predictions))
print('The confusion matrix looks as follows', cf_mat(ground_truth, predictions))